In [9]:
import time
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

In [20]:
PATH_TO_CFG = "./TensorFlow/workspace/training_demo/exported-models/my_model/pipeline.config"
PATH_TO_CKPT = "./TensorFlow/workspace/training_demo/exported-models/my_model/checkpoint"
PATH_TO_LABELS = "./TensorFlow/workspace/training_demo/annotations/label_map.pbtxt"
image_path = "./TensorFlow/workspace/training_demo/images/test/africa-south-africa-wild-nature-67553w.jpeg"

In [16]:
print('Loading model... ', end='')
start_time = time.time()
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

@tf.function
def detect_fn(image):
    """Detect objects in image."""

    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)

    return detections

end_time = time.time()

elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model... Done! Took 0.016023874282836914 seconds


In [17]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS,
                                                                    use_display_name=True)

In [43]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import matplotlib.image as mpimg

In [38]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

In [46]:
print('Running inference for {}... '.format(image_path), end='')

image_np = load_image_into_numpy_array(image_path)
img = mpimg.imread(image_path)

plt.imshow(img)

input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)

detections = detect_fn(input_tensor)

num_detections = int(detections.pop('num_detections'))
detections = {key: value[0, :num_detections].numpy()
                  for key, value in detections.items()}
detections['num_detections'] = num_detections

    # detection_classes should be ints.
detections['detection_classes'] = detections['detection_classes'].astype(np.int64)
label_id_offset = 1
image_np_with_detections = image_np.copy()
detection_scores = detections['detection_scores']
num_scores_over_threshold = (detection_scores > 0.8).sum()

print('{} detections with score over 0.8'.format(num_scores_over_threshold))
detection_boxes = detections['detection_boxes']
for i in range(num_scores_over_threshold):
    if detection_scores[i] > 0.8:
        ymin, xmin, ymax, xmax = detection_boxes[i]
        print('Box {}: ymin={}, xmin={}, ymax={}, xmax={}'.format(i, ymin, xmin, ymax, xmax))


viz_utils.visualize_boxes_and_labels_on_image_array(
            image_np,
            detections['detection_boxes'],
            detections['detection_classes']+label_id_offset,
            detections['detection_scores'],
            category_index,
            use_normalized_coordinates=True,
            max_boxes_to_draw=200,
            min_score_thresh=.10,
            agnostic_mode=False)

plt.figure()
plt.imshow(image_np)
plt.show()
print('Done')
plt.show()

Running inference for ./TensorFlow/workspace/training_demo/images/test/africa-south-africa-wild-nature-67553w.jpeg... 1 detections with score over 0.8
Box 0: ymin=0.49886322021484375, xmin=0.6808141469955444, ymax=0.6448333263397217, xmax=0.7779921293258667
Done
